# Feature selectiojn

sklearn.linear_model.LogisticRegression  fit 한 후에 coef 로 내림차순 정렬 
lightgbm import LGBMClassifier로 fit 한 후에 feature_importances_ 로 내림차순 정렬 
boruta에서서 ranking 내림차순 정렬해서 사용
pymrmr.mRMR(df, 'MIQ', 10)    https://github.com/fbrundu/pymrmr
                               http://home.penglab.com/proj/mRMR/
                               
https://github.com/smazzanti/mrmr/blob/main/mrmr/main.py     f-stat/rf   pearson, user_defined       

# Data loading

* first top 5 is important

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

# load X and y
# NOTE BorutaPy accepts numpy arrays only, hence the .values attribute
X_df = pd.read_csv('test_X.csv', index_col=0)
X = X_df.values
y = pd.read_csv('test_y.csv', header=None, index_col=0).values
y = y.ravel()

In [6]:
import numpy as np

ranking = pd.DataFrame(index = range(X.shape[1]))

# by relevant only 

* sklearn.feature_selection.f_classif(X, y)  by f-stat
* sklearn.feature_selection.mutual_info_classif  by MI


In [8]:
from sklearn.feature_selection import f_classif

f = f_classif(X, y)[0]   # f-statistics

In [9]:
pd.Series(f, index = ranking.index).fillna(0).rank(ascending = False)

0     1.0
1     3.0
2     4.0
3     5.0
4     2.0
5     8.0
6     6.0
7    10.0
8     9.0
9     7.0
dtype: float64

In [12]:
from sklearn.feature_selection import mutual_info_classif

mi = mutual_info_classif(X, y)

In [13]:
pd.Series(mi, index = ranking.index).fillna(0).rank(ascending = False)

0    3.0
1    1.0
2    4.0
3    5.0
4    2.0
5    9.0
6    7.0
7    6.0
8    9.0
9    9.0
dtype: float64

In [18]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier().fit(X, y)

In [21]:
pd.Series(lgbm.feature_importances_, index = ranking.index).rank(ascending = False)

0     5.0
1     1.0
2     2.0
3     3.0
4     8.0
5     6.0
6     9.0
7    10.0
8     4.0
9     7.0
dtype: float64

# mRMR  

* https://github.com/smazzanti/mrmr


<code>
Args:
        X: (pandas.DataFrame) A Dataframe consisting of numeric features only.
        y: (pandas.Series) A Series containing the (categorical) target variable.
        K: (int) Number of features to select.
        relevance: (str or function) Relevance method. 
            If function, it should take X and y as input and return a pandas.Series containing a (non-negative) score of relevance for each feature of X.
            If string, name of method, supported: 'f' (f-statistic), 'rf' (random forest).
        redundancy: (str or function) Redundancy method. 
            If function, it should take A and b as input and return a pandas.Series containing a (non-negative) score of redundancy for each feature of A.
            If string, name of method, supported: 'c' (Pearson correlation)
        denominator: (str or function) Synthesis function to apply to the denominator of MRMR score.
            If function, it should take an iterable as input and return a scalar.
            If string, name of method, supported: 'max', 'mean'
        cat_features: (list) List of categorical features. If None, all string columns will be encoded
        cat_encoding: (str) Name of categorical encoding. Supported: 'leave_one_out', 'james_stein', 'target'
        only_same_domain: (bool) If False, all the necessary correlation coefficients are computed.
            If True, only features belonging to the same domain are compared.
            Domain is defined by the string preceding the first underscore:
            for instance "cusinfo_age" and "cusinfo_income" belong to the same domain,
            whereas "age" and "income" don't.
'''           
</code>

In [27]:
import pandas as pd
from mrmr import mrmr_classif

mrmr = mrmr_classif(pd.DataFrame(X), pd.Series(y), relevance = 'f', redundancy = 'c', K = 784)   # f-test, pearson

100%|██████████| 10/10 [00:04<00:00,  2.48it/s]


In [28]:
pd.Series(list(range(1, len(mrmr) + 1)) + [X.shape[1]] * (X.shape[1] - len(mrmr)), index = mrmr + list(set(ranking.index) - set(mrmr))).sort_index()

0     1
1     3
2     4
3     5
4     2
5     8
6     6
7    10
8     9
9     7
dtype: int64

In [29]:
mrmr = mrmr_classif(pd.DataFrame(X), pd.Series(y), relevance = 'rf', redundancy = 'c', K = 784)   # RF, pearson

100%|██████████| 10/10 [00:04<00:00,  2.02it/s]


In [30]:
pd.Series(list(range(1, len(mrmr) + 1)) + [X.shape[1]] * (X.shape[1] - len(mrmr)), index = mrmr + list(set(ranking.index) - set(mrmr))).sort_index()

0     4
1     1
2     6
3     7
4     3
5     5
6     9
7    10
8     2
9     8
dtype: int64

# PyMRMR

* https://github.com/fbrundu/pymrmr
  * MIQ, MID


In [ ]:
import pymrmr

pymrmr.mRMR(df, 'MIQ', 10)